## libraries used in this assignemnt

In [15]:
# initialize queue length
# queue_length = 0

import numpy as np # for random number distributions
import pandas as pd # for event_log data frame
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 10)
import queue # add FIFO queue data structure
from functools import partial, wraps

import simpy # discrete event simulation environment

## Functions and methods

### Modified Arrival - To represent people leaving the queue based on the patience measurement

In [16]:
def arrival(env, caseid, caseid_queue, event_log):
    # generates new customers... arrivals
    caseid = 0
    while True:  # infinite loop for generating arrivals
        # get the service process started
        # must have waiting customers to begin service
    
        # schedule the time of next customer arrival       
        # by waiting until the next arrival time from now
        # when the process yields an event, the process gets suspended
        inter_arrival_time = round(60*np.random.exponential(scale = mean_inter_arrival_time))
        print("NEXT ARRIVAL TIME: ", env.now + inter_arrival_time)

        yield env.timeout(inter_arrival_time)  # generator function waits
        # env.timeout does not advance the clock. when an event/process calls env.timeout, 
        # that process waits until the clock gets to that time, it does not block 
        # other agents from doing stuff. Other processes can still do stuff 
        # while the first process is waiting for env.timeout to finish.
        caseid += 1
        time = env.now
        activity = 'arrival'
        env.process(event_log_append(env, caseid, time, activity, event_log)) 
        yield env.timeout(0) 
        if caseid_queue.qsize() < balking_queue_length:
            caseid_queue.put(caseid)          
            print("Customer joins queue caseid =",caseid,'time = ',env.now,'queue_length =',caseid_queue.qsize())
            time = env.now
            activity = 'join_queue'
            env.process(event_log_append(env, caseid, time, activity, event_log)) 
            env.process(service_process(env,  caseid_queue, event_log))       
        else:
            print("Customer balks caseid =",caseid,'time = ',env.now,'queue_length =',caseid_queue.qsize()) 
            env.process(event_log_append(env, caseid, env.now, 'balk', event_log)) 

### Serivce process

In [17]:
# user-defined function for random service time (modified exponential)
# returns real number of minutes
def random_service_time(minimum_service_time,mean_service_time,maximum_service_time) :
    try_service_time = np.random.exponential(scale = mean_service_time)
    if (try_service_time < minimum_service_time):
        return(minimum_service_time)
    if (try_service_time > maximum_service_time):
        return(maximum_service_time)
    if (try_service_time >= minimum_service_time) and (try_service_time <= maximum_service_time):
        return(try_service_time)
    


def service_process(env, caseid_queue, event_log):
    # must have baristas to provide service... freeze until request can be met
    with available_baristas.request() as req:
        yield req  # wait until the request can be met.. there must be an available barista
        queue_length_on_entering_service = caseid_queue.qsize()
        caseid = caseid_queue.get()
        print("Begin_service caseid =",caseid,'time = ',env.now,'queue_length =',queue_length_on_entering_service)
        env.process(event_log_append(env, caseid, env.now, 'begin_service', event_log)) 
        # schedule end_service event based on service_time
        service_time = round(60*random_service_time(minimum_service_time,mean_service_time,maximum_service_time))   
        yield env.timeout(service_time)  # sets begin_service as generator function
        queue_length_on_leaving_service = caseid_queue.qsize()
        print("End_service caseid =",caseid,'time = ',env.now,'queue_length =',queue_length_on_leaving_service)
        env.process(event_log_append(env, caseid, env.now, 'end_service', event_log))

### Tracing of the simulation

In [18]:

'''        
---------------------------------------------------------
set up event tracing of all simulation program events 
controlled by the simulation environment
that is, all timeout and process events that begin with "env."
documentation at:
  https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing
  https://docs.python.org/3/library/functools.html#functools.partial
'''
def trace(env, callback):
     """Replace the ``step()`` method of *env* with a tracing function
     that calls *callbacks* with an events time, priority, ID and its
     instance just before it is processed.
     note: "event" here refers to simulaiton program events

     """
     def get_wrapper(env_step, callback):
         """Generate the wrapper for env.step()."""
         @wraps(env_step)
         def tracing_step():
             """Call *callback* for the next event if one exist before
             calling ``env.step()``."""
             if len(env._queue):
                 t, prio, eid, event = env._queue[0]
                 callback(t, prio, eid, event)
             return env_step()
         return tracing_step

     env.step = get_wrapper(env.step, callback)

def trace_monitor(data, t, prio, eid, event):
     data.append((t, eid, type(event)))

def test_process(env):
     yield env.timeout(1)

'''
---------------------------------------------------------
set up an event log for recording events
as defined for the discrete event simulation
we use a list of tuples for the event log
documentation at:
  https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing
'''     
def event_log_append(env, caseid, time, activity, event_log):
    event_log.append((caseid, time, activity))
    yield env.timeout(0)
         

### Parameters to be changed for each simulation

In [19]:
# let the simulation time unit be one second, but set parameters in minutes
# set global variables that do not change throughout the course of the simulation

baristas = 3  # number of barista server resources... vary from 1 to 3

minimum_service_time = 1 # minutes (60 seconds)
mean_service_time = 2 # minutes (120 seconds)
maximum_service_time = 5 # minutes (300 seconds)

mean_inter_arrival_time = 1 # vary from 1 to 10 minutes (60 to 600 seconds)

balking_queue_length = 10  # longest queue length that customers will tolerate

# this binary toggle allows you to fix the random number seed
# so that every run of the program with specific parameter settings
# will yield the same results... setting to false will allow the
# program to obtain different results with each run
obtain_reproducible_results = True

# set length of simulation in simulation time units (assume seconds)
# run for similation_hours of simulation time
simulation_hours = 10
fixed_simulation_time =  simulation_hours*60*60   

parameter_string_list = [str(simulation_hours),'hours',
              str(baristas),str(minimum_service_time),
              str(mean_service_time),str(maximum_service_time),
              str(mean_inter_arrival_time),str(balking_queue_length)]
separator = '-'        
simulation_file_identifier = separator.join(parameter_string_list)

# initialize queue length
# queue_length = 0

### Simulation

In [20]:
'''
---------------------------------------
set up the SimPy simulation environment
and run the simulation with monitoring
---------------------------------------
'''
if obtain_reproducible_results: 
    np.random.seed(9999)

# set up simulation trace monitoring for the simulation
data = []
# bind *data* as first argument to monitor()
this_trace_monitor = partial(trace_monitor, data)

env = simpy.Environment()
trace(env, this_trace_monitor)

env.process(test_process(env))

# implement FIFO queue to hold caseid values
caseid_queue = queue.Queue()

# set up limited server/baristas resource
available_baristas = simpy.Resource(env, capacity = baristas)
caseid = -1  # dummy caseid for beginning of simulation
# dummy caseid values will be omitted from the event_log
# prior to analyzing simulation results

# beginning record in event_log list of tuples of the form
# form of the event_log tuple item: (caseid, time, activity)
event_log = [(caseid,0,'null_start_simulation')]
# event_monitor = partial(event_monitor, event_log)
env.process(event_log_append(env, caseid, env.now, 'start_simulation', event_log))
 
# call customer arrival process/generator to begin the simulation
env.process(arrival(env, caseid, caseid_queue, event_log))  

env.run(until = fixed_simulation_time)  # start simulation with monitoring and fixed end-time

NEXT ARRIVAL TIME:  104
Customer joins queue caseid = 1 time =  104 queue_length = 1
NEXT ARRIVAL TIME:  119
Begin_service caseid = 1 time =  104 queue_length = 1
Customer joins queue caseid = 2 time =  119 queue_length = 1
NEXT ARRIVAL TIME:  166
Begin_service caseid = 2 time =  119 queue_length = 1
End_service caseid = 1 time =  164 queue_length = 0
Customer joins queue caseid = 3 time =  166 queue_length = 1
NEXT ARRIVAL TIME:  178
Begin_service caseid = 3 time =  166 queue_length = 1
Customer joins queue caseid = 4 time =  178 queue_length = 1
NEXT ARRIVAL TIME:  234
Begin_service caseid = 4 time =  178 queue_length = 1
End_service caseid = 2 time =  179 queue_length = 0
End_service caseid = 3 time =  232 queue_length = 0
Customer joins queue caseid = 5 time =  234 queue_length = 1
NEXT ARRIVAL TIME:  296
Begin_service caseid = 5 time =  234 queue_length = 1
End_service caseid = 4 time =  259 queue_length = 0
Customer joins queue caseid = 6 time =  296 queue_length = 1
NEXT ARRIVAL